In [4]:
import pandas as pd
import PyPDF2
import fitz
from io import StringIO
import shutil
import os
from pathlib import Path

In [5]:
# This script takes the same annotations in one document of reference

doc = fitz.open('document_with_annotations.pdf')

rect_lists = [[] for i in range(4)] 

for page in doc:
    annots = page.annots()
    for i, annot in enumerate(annots):
        rects = annot.rect
        for rect in rects:
            rect_lists[i].append(rect)

#for rect_list in rect_lists:
    #print(rect_list)


In [40]:
# Here we iterate every document in one folder from Local, draw the annotations and save with different name in another folder

SOURCE_FOLDER = 'SOURCE_FOLDER'
DEST_FOLDER = 'DEST_FOLDER'

for filename in os.listdir(SOURCE_FOLDER):
    doc2 = fitz.open(os.path.join(SOURCE_FOLDER, filename))
    page = doc2[0]
    for rect_list in rect_lists:    
        page.add_rect_annot(rect_list)
    doc2.save(os.path.join(DEST_FOLDER, 'CAPT_' + filename))
    doc2.close()


In [9]:
# Here we iterate every document in one folder from the Cloud, draw the annotations and save with different name in another folder

# Replace the directory path according to your One Drive
onedrive_directory_path = Path('https://onedrive.live.com/?id=51DA451495FD6C4%2128039&cid=051DA451495FD6C4')

# Replace the source and destination folder paths
SOURCE_FOLDER = onedrive_directory_path / 'SOURCE_FOLDER'
DEST_FOLDER = onedrive_directory_path / 'DEST_FOLDER'

# List all files in the source folder
files_in_source = os.listdir(SOURCE_FOLDER)

for filename in files_in_source:
    doc2 = fitz.open(os.path.join(SOURCE_FOLDER, filename))
    page = doc2[0]
    for rect_list in rect_lists:    
        page.add_rect_annot(rect_list)
    doc2.save(os.path.join(DEST_FOLDER, 'CAPT_' + filename))
    doc2.close()

OSError: [WinError 123] El nombre de archivo, el nombre de directorio o la sintaxis de la etiqueta del volumen no son correctos: 'https:\\onedrive.live.com\\?id=51DA451495FD6C4%2128039&cid=051DA451495FD6C4\\SOURCE_FOLDER'

In [41]:
def make_text(words):
    line_dict = {} 
    words.sort(key=lambda w: w[0])
    for w in words:  
        y1 = round(w[3], 1)  
        word = w[4] 
        line = line_dict.get(y1, [])  
        line.append(word)  
        line_dict[y1] = line  
    lines = list(line_dict.items())
    lines.sort()  
    return "n".join([" ".join(line[1]) for line in lines])

In [42]:
pol_dic = {'NO.POLIZA': [],
           'NOMBRE': [], 
           'RFC': [], 
           'FORMA PAGO': []}

for filename in os.listdir(DEST_FOLDER):
    doc2 = fitz.open(os.path.join(DEST_FOLDER, filename))
    page = doc2[0]
    all_annots = []
    for annot in page.annots():
        if annot!=None:
            rec = annot.rect
            words = page.get_text("words") 
            mywords = [w for w in words if fitz.Rect(w[:4]) in rec]
            ann = make_text(mywords)
            all_annots.append(ann)
    pol_dic['NO.POLIZA'].append(all_annots[2])
    pol_dic['NOMBRE'].append(all_annots[0])
    pol_dic['RFC'].append(all_annots[1])
    pol_dic['FORMA PAGO'].append(all_annots[3])


In [43]:
pol_dic

{'NO.POLIZA': ['150202988300',
  '150203048000',
  '150203224600',
  '150203649700',
  '150203760000'],
 'NOMBRE': ['CORNELIO URREA MONTES',
  'JOSE ALFONSO TOLEDO LICEA',
  'PABLO ANTONIO HERRERA',
  'HECTOR AVENA LOPEZ',
  'VICTORIA EUGENIA DIAZ RUIZ'],
 'RFC': ['UEMC730326XXX',
  'TOLA840813XXX',
  'HECP911101XXX',
  'AELH750114XXX',
  'DIRV540518000'],
 'FORMA PAGO': ['Contado', 'Contado', 'Contado', 'Contado', 'Contado']}

In [44]:
pol_data = pd.DataFrame.from_dict(pol_dic)


In [45]:
pol_data

,NO.POLIZA,NOMBRE,RFC,FORMA PAGO
0,150202988300,CORNELIO URREA MONTES,UEMC730326XXX,Contado
1,150203048000,JOSE ALFONSO TOLEDO LICEA,TOLA840813XXX,Contado
2,150203224600,PABLO ANTONIO HERRERA,HECP911101XXX,Contado
3,150203649700,HECTOR AVENA LOPEZ,AELH750114XXX,Contado
4,150203760000,VICTORIA EUGENIA DIAZ RUIZ,DIRV540518000,Contado


In [46]:
pol_data.to_excel('pol_final.xlsx', index=False)

In [19]:
# On this script we can eliminate the original files after saving into the new folder 

for root, dirs, files in os.walk(DEST_FOLDER):
    for file in files:
        if file.startswith("CAPT_"):
            path = os.path.join(root, file)
            file_name = file.replace("CAPT_", "")
            if os.path.exists(os.path.join(SOURCE_FOLDER, file_name)):
                os.remove(os.path.join(SOURCE_FOLDER, file_name))